<a href="https://colab.research.google.com/github/granatb/02456-deep-learning-with-PyTorch/blob/master/pipeline_case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable 
from torch.utils.data import TensorDataset, DataLoader


sns.set_style("whitegrid")
%matplotlib inline

In [ ]:
def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [ ]:
def Lag_N(series, N):
    return series.shift(N)

def Lag_diff(series, N):
    return series.diff(periods = N)

def Lag_ratio(series, N):
    return series/series.shift(N)

def AVG_N(series, N):
    ret = [0]*len(series)
    for i in range(N,len(series)):
        ret[i] = np.mean(series[i-N:i])
    return ret

def Min_N(series, N):
    ret = [0]*len(series)
    for i in range(N, len(series)):
        ret[i] = np.min(series[i-N:i])
    return ret

def Max_N(series, N):
    ret = [0]*len(series)
    for i in range(N, len(series)):
        ret[i] = np.max(series[i-N:i])
    return ret

def Avg_shift(series, rng):
    ret = [0]*len(series)
    for i in range(rng[1], len(series)):
        ret[i] = np.mean(series[i-rng[1]:i-rng[0]+1])
    return ret
def SD_N(series, N):
    ret = [0] * len(series)
    for i in range(N, len(series)):
        ret[i] = np.std(series[i - N:i])
    return ret

def EMA_N(series, N):
    ret = pd.Series.ewm(series, span = N, adjust = False).mean()
    return ret

def MACD(series, args):
    ema_1 = pd.Series.ewm(series, span = args[0], adjust = False).mean()
    ema_2 = pd.Series.ewm(series, span = args[1], adjust = False).mean()
    ema_signal = pd.Series.ewm(series, span = args[2], adjust = False).mean()
    ret = ema_1-ema_2-ema_signal
    return ret

def SO_N(series, N):
    ret = [0] * len(series)
    for i in range(N, len(series)):
        L_N = np.min(series[i-N:i])
        H_N = np.max(series[i-N:i])
        ret[i] = 100*(series[i-1]-L_N)/(H_N-L_N)
    return ret
    
def RSI_N(series, N):
    delta = series.diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    avg_gain = [0] * len(series)
    avg_loss = [0] * len(series)
    avg_gain[N+1] = pd.Series(up[1:N+1]).mean()
    avg_loss[N+1] = pd.Series(down[1:N+1]).mean()
    for i in range(N+2, len(series)):
        avg_gain[i] = ((N-1)*avg_gain[i-1]+up[i-1])/N
        avg_loss[i] = ((N-1) * avg_loss[i-1]+down[i-1]) / N

    RS = pd.Series(avg_gain) / ((-1) * pd.Series(avg_loss))
    RSI = 100.0 - (100.0 / (1.0 + RS))

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
datasets_path = "/content/drive/My Drive/data/Datasets/"

In [ ]:
df = pd.read_csv(datasets_path+'Case3/Proj_NWP_Case3.csv')
df.Date_Time = pd.to_datetime(df.Date_Time)

In [ ]:
dfm = pd.read_csv(datasets_path+'Case3/Proj_Measurements_Case3.csv')
dfm.Date_Time = pd.to_datetime(dfm.Date_Time)

In [ ]:
# df_full = dfm.add_suffix('_m').join(df.add_suffix('_nwp'))
df_full = dfm.merge(df, on = "Date_Time", suffixes=('_m', '_nwp'))
df_full['Month'] = pd.DatetimeIndex(df_full.Date_Time).month
df_full['Year'] = pd.DatetimeIndex(df_full.Date_Time).year
df_full['Day'] = pd.DatetimeIndex(df_full.Date_Time).day
df_full.head(3)

In [ ]:
target_variable = ['Park Power [KW]']
model_cols = ['Date_Time', 'Month', 'Year', 'Speed_100m', 'Direction_100m','Air Density_100m','Speed_50m_nwp', 'Direction_50m_nwp','Air Density_50m','Speed_10m_nwp','Speed_150m']
final_cols = ['Speed_100m', 'Direction_100m','Speed_150m','Speed_50m_nwp', 'Direction_50m_nwp','Air Density_50m','Speed_10m_nwp']

In [ ]:
df_model = df_full[target_variable+model_cols]
df_model.dropna(inplace=True)
df_model = df_model[~(df_model['Park Power [KW]'] < 0)]
df_model = df_model[~((df_model.Speed_100m > 4) & (df_model['Park Power [KW]'] == 0))]
df_model = df_model[~((df_model.Speed_10m_nwp > 4) & (df_model['Park Power [KW]'] == 0))]
df_model = df_model[~((df_model.Speed_50m_nwp > 4) & (df_model['Park Power [KW]'] == 0))]
df_model['Power_SD_12'] = SD_N(df_model['Park Power [KW]'], 12)
df_model.loc[df_model.Speed_100m > 12, 'Speed_100m'] = 12
df_model.loc[df_model.Speed_10m_nwp > 7.12, 'Speed_10m_nwp'] = 7.12
df_model.loc[df_model.Speed_50m_nwp > 6.16, 'Speed_50m'] = 6.16
df_model = df_model[~(df_model['Power_SD_12'] == 0)]
df_model.set_index('Date_Time', inplace = True)
df_model = df_model.loc[:, target_variable+final_cols]
df_model.shape

In [ ]:
df_model.head()

In [ ]:
y = df_model['Park Power [KW]']
X = df_model[final_cols]

In [ ]:
sequence_length = 16

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
mm = MinMaxScaler()
ss = StandardScaler()


X_ss = ss.fit_transform(X)
y_mm = mm.fit_transform(y.to_numpy().reshape(-1, 1))
y_mm = y_mm[sequence_length:]

In [ ]:
X_seq = []
for i in range(0, X.shape[0]):
    if i >= sequence_length:
        i_start = i - sequence_length + 1
        x = X_ss[i_start:(i + 1), :]
        X_seq.append(x)

In [ ]:
X_seq = np.asarray(X_seq)

In [ ]:
from sklearn.model_selection import train_test_split
# def train_val_test_split(X, y, target_col, test_ratio):
#     val_ratio = test_ratio / (1 - test_ratio)
#     # X, y = feature_label_split(df, target_col)
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)
#     X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)
#     return X_train, X_val, X_test, y_train, y_val, y_test

# X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X_seq, y_mm, 'value', 0.2)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_mm, test_size=0.2, shuffle=True)

In [ ]:
print("Training Shape", X_train.shape, y_train.shape)
print("Testing Shape", X_test.shape, y_test.shape) 

In [ ]:
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test)) 

In [ ]:
batch_size = 32

train = TensorDataset(X_train_tensors, y_train_tensors)
test = TensorDataset(X_test_tensors, y_test_tensors)

# train_loader = DataLoader(train, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader_one = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
class LSTM1(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.num_classes = num_classes #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length
        linear_out_size = 64

        # self.linear_out =  nn.Linear(input_size, linear_out_size) #fully connected 1

        self.fnn = nn.Sequential(
            nn.Linear(input_size, linear_out_size, bias=False),
            nn.BatchNorm1d(linear_out_size),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(linear_out_size, linear_out_size, bias=False)
        )

        self.conv2_drop = nn.Dropout2d()

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        # self.fc_1 =  nn.Linear(hidden_size+linear_out_size, 256) #fully connected 1
        self.fc_1 = nn.Sequential(
            nn.Linear(hidden_size+linear_out_size, 256, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 256, bias=False)
        )
        self.fc = nn.Linear(256, num_classes) #fully connected last layer

        self.relu = nn.ReLU()
    
    def forward(self,x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        # print(h_0.shape)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # lin_out = self.linear_out(x[:,-1])
        lin_out = self.fnn(x[:,-1])
        lin_out = self.relu(lin_out)
        # print(lin_out.shape)
        # print(x.shape)
        output, (hn, cn) = self.lstm(x.to(device), (h_0.to(device), c_0.to(device))) #lstm with input, hidden, and internal state
        # print(hn.shape)
        hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        out = self.relu(hn)
        # print(out.shape)
        out =  torch.cat((out, lin_out),1)
        # print(out.shape)
        out = self.fc_1(out) #first Dense
        out = self.relu(out) #relu
        out = self.fc(out) #Final Output
        out = self.relu(out)
        return out

In [ ]:
class Optimization:
    def __init__(self, model, loss_fn, optimizer):
        self.model = model
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_losses = []
        self.val_losses = []
    
    def train_step(self, x, y):
        # Sets model to train mode
        self.model.train()

        # Makes predictions
        yhat = self.model(x)

        # Computes loss
        loss = self.loss_fn(y, yhat)

        # Computes gradients
        loss.backward()

        # Updates parameters and zeroes gradients
        self.optimizer.step()
        self.optimizer.zero_grad()

        # Returns the loss
        return loss.item()

    def train(self, train_loader, val_loader, batch_size=64, n_epochs=50, n_features=1):
        model_path = f'models/{self.model}_{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}'
          # Define the K-fold Cross Validator
 


        for epoch in range(1, n_epochs + 1):
            batch_losses = []
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.view([batch_size, -1, n_features]).to(device)
                y_batch = y_batch.to(device)
                loss = self.train_step(x_batch, y_batch)
                batch_losses.append(loss)
            training_loss = np.mean(batch_losses)
            self.train_losses.append(training_loss)

            with torch.no_grad():
                batch_val_losses = []
                for x_val, y_val in val_loader:
                    x_val = x_val.view([batch_size, -1, n_features]).to(device)
                    y_val = y_val.to(device)
                    self.model.eval()
                    yhat = self.model(x_val)
                    val_loss = self.loss_fn(y_val, yhat).item()
                    batch_val_losses.append(val_loss)
                validation_loss = np.mean(batch_val_losses)
                self.val_losses.append(validation_loss)

            if (epoch <= 10) | (epoch % 50 == 0):
                print(
                    f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
                )

            # torch.save(self.model.state_dict(), model_path)


    def evaluate(self, test_loader, batch_size=1, n_features=1):
        with torch.no_grad():
            predictions = []
            values = []
            for x_test, y_test in test_loader:
                x_test = x_test.view([batch_size, -1, n_features]).to(device)
                y_test = y_test.to(device)
                self.model.eval()
                yhat = self.model(x_test)
                predictions.append(yhat.cpu().numpy()) #.to(device).detach()
                values.append(y_test.cpu().numpy())

        return predictions, values

    def plot_losses(self):
        plt.plot(self.train_losses, label="Training loss")
        plt.plot(self.val_losses, label="Validation loss")
        plt.legend()
        plt.title("Losses")
        plt.show()
        plt.close()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.optim as optim
from datetime import datetime
input_dim = 7 #len(X_train.columns)
output_dim = 1
hidden_dim = 64
layer_dim = 5
batch_size = 64
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim' : hidden_dim,
                'layer_dim' : layer_dim,
                'output_dim' : output_dim,
                'batch_size':batch_size}

input_size = 7 #number of features
hidden_size = 64 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers

num_classes = 1 #number of output classes 
cv = False


if cv:

    kfold = KFold(n_splits=5, shuffle=True)

        # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(train)):
        
        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')
        
        model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]).to(device) #our lstm class 

        # for layer in self.model.children():
        #     if hasattr(layer, 'reset_parameters'):
        #         layer.reset_parameters()


        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
        
        # Define data loaders for training and testing data in this fold
        train_loader = torch.utils.data.DataLoader(
                        train, 
                        batch_size=64, sampler=train_subsampler, drop_last=True)
        val_loader = torch.utils.data.DataLoader(
                        train,
                        batch_size=64, sampler=test_subsampler, drop_last = True)   
        
        loss_fn = nn.MSELoss(reduction="mean")
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
        opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
        opt.plot_losses()

        predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

else:

    model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]).to(device) #our lstm class 

    # for layer in self.model.children():
    #     if hasattr(layer, 'reset_parameters'):
    #         layer.reset_parameters()


    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(list(range(80000)))
    test_subsampler = torch.utils.data.SubsetRandomSampler(list(range(80000, len(train))))
    
    # Define data loaders for training and testing data in this fold
    train_loader = torch.utils.data.DataLoader(
                    train, 
                    batch_size=64, sampler=train_subsampler, drop_last=True)
    val_loader = torch.utils.data.DataLoader(
                    train,
                    batch_size=64, sampler=test_subsampler, drop_last = True)   
    
    loss_fn = nn.MSELoss(reduction="mean")
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
    opt.train(train_loader, val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
    opt.plot_losses()

    predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)
# model = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors.shape[1]).to(device) #our lstm class 

# loss_fn = nn.MSELoss(reduction="mean")
# optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# opt = Optimization(model=model, loss_fn=loss_fn, optimizer=optimizer)
# opt.train(train, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
# opt.plot_losses()

# predictions, values = opt.evaluate(test_loader_one, batch_size=1, n_features=input_dim)

In [ ]:
def inverse_transform(scaler, df, columns):
    for col in columns:
        df[col] = scaler.inverse_transform(df[col])
    return df


def format_predictions(predictions, values, df_test, scaler):
    vals = np.concatenate(values, axis=0).ravel()
    preds = np.concatenate(predictions, axis=0).ravel()
    df_result = pd.DataFrame(data={"value": vals, "prediction": preds})
    df_result = df_result.sort_index()
    df_result = inverse_transform(scaler, df_result, [["value", "prediction"]])
    return df_result


df_result = format_predictions(predictions, values, X_test, mm)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(df):
    return {'mae' : mean_absolute_error(df.value, df.prediction),
            'rmse' : mean_squared_error(df.value, df.prediction) ** 0.5,
            'r2' : r2_score(df.value, df.prediction)}

result_metrics = calculate_metrics(df_result)
result_metrics

In [ ]:
train_predict = model(X_test_tensors.to(device))#forward pass
data_predict = train_predict.cpu().data.numpy() #numpy conversion
dataY_plot = y_test

data_predict = mm.inverse_transform(data_predict) #reverse transformation
dataY_plot = mm.inverse_transform(dataY_plot)
plt.figure(figsize=(10,6)) #plotting
# plt.axvline(x=100000, c='r', linestyle='--') #size of the training set

plt.plot(dataY_plot, label='Actuall Data') #actual plot
plt.plot(data_predict, label='Predicted Data') #predicted plot
plt.title('Time-Series Prediction')
plt.legend()
plt.show() 

In [ ]:
fig , axs = plt.subplots(6, 4, figsize=(32, 40))
axs = axs.flatten()
i = 0
for i, ax in zip(list(range(25)), axs):
    ax.plot(dataY_plot[i*1000:(i+1)*1000], label='Actuall Data', alpha=0.7)
    ax.plot(data_predict[i*1000:(i+1)*1000], label='Predicted Data') #predicted plot

    # ax.axis('off')
    ax.title.set_text(f'iter: {i}')
    i += 1

plt.show()

# Save the model

In [ ]:
checkpoint = {'model_params': model_params,
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint_case3.pth')

# Load the model

In [ ]:
# chk = torch.load('checkpoint_case2.pth')
# print(chk.keys())

In [ ]:
# model.load_state_dict(chk['state_dict'])

# Explanations

In [ ]:
# import shap

In [ ]:
# X_train.shape

In [ ]:
# e = shap.DeepExplainer(
#         model, 
#         X_train_tensors.to(device)
# )

In [ ]:
# shap_values = e.shap_values(
#     X_train_tensors[1:10].to(device)
# )

In [ ]:
# df = pd.DataFrame({
#     "mean_abs_shap": np.mean(np.abs(shap_values), axis=0), 
#     "stdev_abs_shap": np.std(np.abs(shap_values), axis=0), 
#     "name": final_cols
# })
# df.sort_values("mean_abs_shap", ascending=False)

# Daily prediction

In [ ]:
df = pd.read_csv(datasets_path+'Day2NWP_C3.csv')
df.Date_Time = pd.to_datetime(df.Date_Time)

In [ ]:
df.rename(columns={'Speed_10m':'Speed_10m_nwp', 'Speed_50m':'Speed_50m_nwp', 'Direction_50m':'Direction_50m_nwp'},inplace=True)

In [ ]:
# final_cols = ['Speed_100m', 'Direction_100m','Air Density_100m','Speed_50m', 'Direction_50m','Air Density_50m','Speed_10m_nwp']
X_pred = df[final_cols]

In [ ]:
X_ss_pred = ss.transform(X_pred)

In [ ]:
X_seq_pred = []
for i in range(0, X_ss_pred.shape[0]):
    if i >= sequence_length:
        i_start = i - sequence_length + 1
        x = X_ss_pred[i_start:(i + 1), :]
        X_seq_pred.append(x)

In [ ]:
X_seq_pred = np.asarray(X_seq_pred)[-96:]

In [ ]:
X_seq_pred.shape

In [ ]:
X_pred_tensor = Variable(torch.Tensor(X_seq_pred))

In [ ]:
train_predict = model(X_pred_tensor.to(device))
data_predict = train_predict.cpu().data.numpy() #numpy conversion
data_predict = mm.inverse_transform(data_predict)

In [ ]:
res = pd.DataFrame(data_predict, columns=['power_pred_c3'])
res.set_index(df['Date_Time'][-96:],inplace=True)

In [ ]:
res.to_csv('day2_case3_solobolo.csv')

In [ ]:
res